In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import tldextract
import urllib

import sys
sys.path.append("/scratch/venia/web2wiki/helpers")


In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
import pyarrow.parquet as pq
import pyarrow as pa

from bs4 import BeautifulSoup

import os


from settings import DATA_DIR, EMBEDDING_DIR


os.environ['SPARK_HOME'] = "/home/veselovs/spark-3.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/veselovs/jdk-13.0.2"

config = pyspark.SparkConf().setAll([('spark.executor.memory', '130g'),
                                 ('spark.executor.cores', '40'),
                                 ('spark.driver.memory','130g'),
                                 ('spark.driver.maxResultSize','0'),
                                 ('spark.python.worker.memory', '15g'),
                                 ('spark.reducer.maxSizeInFlight','15g'),
                                 ('spark.rpc.message.maxSize', '1000'),
                                 ('spark.sql.autoBroadcastJoinThreshold','-1'),
                                 ('spark.local.dir', '/tmp/')])
sc = pyspark.SparkContext(conf=config)
spark = SparkSession.builder.appName('Venia').config(conf=config).getOrCreate()



22/08/18 16:24:34 WARN Utils: Your hostname, iccluster039 resolves to a loopback address: 127.0.1.1; using 10.90.38.15 instead (on interface ens786f0)
22/08/18 16:24:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/18 16:24:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/18 16:24:35 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/08/18 16:24:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.load("/scratch/venia/web2wiki/data/all_url_wiki_embedding.parquet")

In [4]:
def convert_to_dict(x):
    from collections import Counter
    try:
        x = dict(eval(x))
    except:
        x = x
    return x

convert_to_dict_udf = F.udf(convert_to_dict,MapType(StringType(), FloatType()))


In [5]:
df

DataFrame[url: string, domain: string, wiki_link: string, embedding: string, title: string]

In [6]:
f = df.select("url","domain","wiki_link","title", convert_to_dict_udf("embedding").alias("dict"))

In [7]:
from collections import Counter

In [8]:
ff["dict"].iloc[0]

NameError: name 'ff' is not defined

In [9]:
vals = f.select("url","domain","wiki_link","title",F.explode("dict"))

In [10]:
top_100 = vals.groupBy("key").count().orderBy("count",ascending=False).limit(100)

In [11]:
vals2 = vals.join(top_100, on = "key",how = "inner")

In [12]:
vals3 = vals2.groupBy("url","domain","wiki_link","title").pivot("key").mean("value")

In [ ]:
vals3.write.parquet("/scratch/venia/web2wiki/data/integrated_use.parquet")

22/08/18 05:26:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [64]:
vals4 = spark.read.load("/scratch/venia/web2wiki/data/integrated_use.parquet")

In [65]:
vals4 = vals4.limit(1000000).fillna(0)

In [7]:
vals4 = vals4.cache()

22/08/18 16:26:02 WARN CacheManager: Asked to cache already cached data.


In [56]:
crawl_dump =spark.read.option("delimiter", "\t").csv(DATA_DIR + "common_crawl_dump/cc-main-2021-feb-apr-may-domain-ranks.txt.gz", header = True)



In [57]:
# this fn is necessary for reversing the host names
@F.udf 
def reverse_reverse_host(x):
    x = x.split(".")
    x = x[-1::-1]
    x = ".".join(x)
    return x

crawl_dump = crawl_dump.withColumn("domain",reverse_reverse_host("#host_rev"))

In [58]:
crawl_dump = crawl_dump.withColumn("#pr_pos", F.col("#pr_pos").cast(IntegerType()))

In [59]:
crawl_dump = crawl_dump.orderBy("#pr_pos", ascending=True).limit(100000)

In [ ]:
crawl_dump.write.parquet("/scratch/venia/web2wiki/data/top_domains_pr.parquet", mode = "overwrite")

In [61]:
crawl_dump2 = spark.read.load("/scratch/venia/web2wiki/data/top_domains_pr.parquet")

In [66]:
vals5 = vals4.join(crawl_dump2, on = "domain", how = "inner")

In [67]:

from sklearn.metrics import pairwise

In [68]:
vals6 = vals5.toPandas()

In [70]:
vals6

,domain,url,wiki_link,title,[document],a,address,ahref,align,app-root,...,u,ul,wbr,whatuseek_winona_noindex,#harmonicc_pos,#harmonicc_val,#pr_pos,#pr_val,#host_rev,#n_hosts
0,3qdigital.com,http://3qdigital.com/blog/affiliate-marketing-...,http://en.wikipedia.org/wiki/Aral_Sea,Aral Sea,0.001862,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,43083,1.7447698E7,43565,8.6262567847913E-7,com.3qdigital,2
1,allthingsd.com,http://allthingsd.com/20131105/ipad-air-has-sp...,http://en.wikipedia.org/wiki/Cyclic_olefin_cop...,Cyclic olefin copolymer,0.001637,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2164,1.8598746E7,4918,7.5253717150172684E-6,com.allthingsd,25
2,allthingsd.com,http://allthingsd.com/20120914/twitters-loss-i...,http://en.wikipedia.org/wiki/Hobson's_choice,Hobson's choice,0.001704,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2164,1.8598746E7,4918,7.5253717150172684E-6,com.allthingsd,25
3,alvinalexander.com,http://alvinalexander.com/blog/post/technology...,http://en.wikipedia.org/wiki/Digital_rights_ma...,Digital rights management,0.002392,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,336971,1.5324381E7,80099,4.739542258562919E-7,com.alvinalexander,1
4,alvinalexander.com,http://alvinalexander.com/misc/you-neither-rig...,https://en.wikipedia.org/wiki/Benjamin_Graham,Benjamin Graham,0.003003,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,336971,1.5324381E7,80099,4.739542258562919E-7,com.alvinalexander,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33781,volokh.com,http://volokh.com/category/popular-culture,http://en.wikipedia.org/wiki/Elementary_(TV_se...,Elementary (TV series),0.000648,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25870,1.7569518E7,47966,7.79928560211914E-7,com.volokh,2
33782,volokh.com,http://volokh.com/category/sunday-song,http://en.wikipedia.org/wiki/Blows_Against_the...,Blows Against the Empire,0.000617,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25870,1.7569518E7,47966,7.79928560211914E-7,com.volokh,2
33783,volokh.com,http://volokh.com/2012/12/29/putin-signs-law-b...,http://en.wikipedia.org/wiki/Human_rights_in_R...,Human rights in Russia,0.000956,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25870,1.7569518E7,47966,7.79928560211914E-7,com.volokh,2
33784,volokh.com,http://volokh.com/category/popular-culture,http://en.wikipedia.org/wiki/Final_Crisis,Final Crisis,0.000648,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25870,1.7569518E7,47966,7.79928560211914E-7,com.volokh,2


In [37]:
mat = pd.DataFrame(pairwise.cosine_similarity(resp), columns = vals6["url"], index=vals6["url"])

In [39]:
resp

array([[-0.09130022,  0.10991695, -0.14438061, -0.05707606, -0.03125145],
       [-0.08478644,  0.10337553, -0.14314442, -0.02732246, -0.01032792],
       [ 0.10005573,  0.0265546 , -0.09731244,  0.01354177,  0.08264192],
       ...,
       [ 0.00772586,  0.04262923, -0.12112827,  0.15585028,  0.1351922 ],
       [-0.00054812,  0.0785278 , -0.11858091, -0.09147737, -0.02354899],
       [ 0.06993391,  0.05418252, -0.098516  , -0.11845397, -0.01781905]])

In [32]:
from sklearn.decomposition import PCA
pca = PCA(5)


In [71]:
sim_dict = {}
grouped = vals6.groupby("domain")

for domain, vals in grouped:
    print(domain)
    print("Length", len(vals))
    if len(vals) > 3:
        domain2 = domain
        mat = pd.DataFrame(pairwise.cosine_similarity(vals[cols]), columns = vals["url"], index=vals["url"])
        sim_dict[domain] = mat

1337x.to
Length 1
1stwebdesigner.com
Length 1
20somethingfinance.com
Length 3
300mpg.org
Length 2
303magazine.com
Length 1
3dprint.com
Length 9
3dprintingindustry.com
Length 3
3qdigital.com
Length 1
3quarksdaily.com
Length 31
411us.info
Length 18
444.hu
Length 12
51degrees.com
Length 1
5by5.tv
Length 10
60secondmarketer.com
Length 1
7news.com.au
Length 1
80000hours.org
Length 2
8thlight.com
Length 4
99bitcoins.com
Length 5
99designs.co.uk
Length 4
99designs.com
Length 4
99percentinvisible.org
Length 6
9to5mac.com
Length 4
9to5toys.com
Length 13
a16z.com
Length 4
aalbc.com
Length 1
aaslh.org
Length 1
abeautifulmess.com
Length 3
able2know.org
Length 14
about.me
Length 8
acestoohigh.com
Length 38
acko.net
Length 1
activerain.com
Length 12
actualitte.com
Length 1
adactio.com
Length 65
adage.com
Length 1
adainitiative.org
Length 4
adamnash.blog
Length 7
adblockplus.org
Length 1
addicted2success.com
Length 10
adevarul.ro
Length 4
adguard.com
Length 2
admiralmarkets.com
Length 1
adrianroselli

cosmosmagazine.com
Length 1
countercurrents.org
Length 10
coupontoaster.co.uk
Length 3
cpdl.org
Length 17
crackberry.com
Length 1
craftcms.com
Length 2
crate.io
Length 1
createaprowebsite.com
Length 1
creation.com
Length 1
creativecommons.org
Length 1
creativecommons.org.au
Length 1
crimethinc.com
Length 5
crookedtimber.org
Length 18
crooksandliars.com
Length 10
crosscut.com
Length 1
crwflags.com
Length 7
cryptography.io
Length 2
cryptomundo.com
Length 4
cryptonews.com
Length 1
cryptoslate.com
Length 2
css-tricks.com
Length 10
ctftime.org
Length 2
cults3d.com
Length 3
cuni.cz
Length 2
cupofjo.com
Length 2
current.org
Length 2
customersthatstick.com
Length 3
cvshealth.com
Length 1
cxl.com
Length 4
cyberbullying.org
Length 1
cybersecurityventures.com
Length 2
cybertelecom.org
Length 1
cybre.space
Length 3
cycling74.com
Length 1
cyclingtips.com
Length 7
cyndislist.com
Length 1
cyware.com
Length 1
dailycaller.com
Length 11
dailyhive.com
Length 1
dailyhodl.com
Length 1
dailystoic.com
Length

hightimes.com
Length 5
hintjens.com
Length 3
hired.com
Length 1
historynewsnetwork.org
Length 3
hitconsultant.net
Length 1
hive.blog
Length 5
hnn.us
Length 2
hollywoodlife.com
Length 1
hostingdata.co.uk
Length 1
hot-topic.co.nz
Length 11
hotair.com
Length 1
hotandbestdeals.com
Length 1
howlround.com
Length 7
howmuch.net
Length 1
howtonode.org
Length 1
htmag.co.il
Length 5
html.cn
Length 1
html.com
Length 2
html5doctor.com
Length 1
htxt.co.za
Length 3
hubpages.com
Length 31
huggingface.co
Length 1
humanevents.com
Length 3
humanists.international
Length 6
hydrogenaud.io
Length 4
hynek.me
Length 1
hypebeast.com
Length 3
hyperallergic.com
Length 66
hypothes.is
Length 83
iapmo.org
Length 1
iatistandard.org
Length 1
iau.org
Length 3
icannwiki.org
Length 4
ico.org.uk
Length 1
idealliance.org
Length 1
identi.ca
Length 1
ifanboy.com
Length 4
iffr.com
Length 1
ifstudies.org
Length 1
iiasa.ac.at
Length 1
iisc.ac.in
Length 3
ij.org
Length 2
ijnet.org
Length 9
ilovetypography.com
Length 1
imagej.ne

moovitapp.com
Length 11
moreniche.com
Length 1
morganlinton.com
Length 2
motamem.org
Length 2
motionographer.com
Length 1
motor.ru
Length 1
mountainx.com
Length 2
moviefit.me
Length 5
moz.com
Length 7
mpora.com
Length 1
mrclay.org
Length 5
mronline.org
Length 5
msmagazine.com
Length 4
mspoweruser.com
Length 1
mtsu.edu
Length 1
muabannhanh.com
Length 1
mugenguild.com
Length 2
multco.us
Length 1
musebycl.io
Length 3
musescore.org
Length 37
museumhack.com
Length 5
musicbrainz.org
Length 12
muzikum.eu
Length 1
my-ecoach.com
Length 2
my-personaltrainer.it
Length 146
myanimelist.net
Length 3
mybroadband.co.za
Length 3
mycountdown.org
Length 27
myfavoritewebdesigns.com
Length 1
myfooddata.com
Length 1
mymodernmet.com
Length 5
mynewsla.com
Length 1
mynorthwest.com
Length 1
mysqlserverteam.com
Length 3
mysteriousuniverse.org
Length 5
mytruemedia.com
Length 3
myx.global
Length 2
n4g.com
Length 1
nachtkritik.de
Length 1
nacla.org
Length 1
naked-science.ru
Length 24
nameexperts.com
Length 4
nanone

soapboxie.com
Length 3
social-hire.com
Length 1
socialcompare.com
Length 1
socialmediaexplorer.com
Length 1
socialnomics.net
Length 1
socratic.org
Length 1
sofiaglobe.com
Length 2
sofrep.com
Length 26
softwareengineeringdaily.com
Length 2
softwarelivre.org
Length 9
sojo.net
Length 2
solace.com
Length 2
solitaired.com
Length 1
somafm.com
Length 1
songexploder.net
Length 7
songmeanings.com
Length 2
sonspring.com
Length 1
soranews24.com
Length 7
sourcehut.org
Length 1
sourcewatch.org
Length 1
spacecoastdaily.com
Length 5
spaceref.com
Length 1
spaceweather.com
Length 3
spannerfilms.net
Length 1
spartacus-educational.com
Length 2
speakerdeck.com
Length 1
speakerpedia.com
Length 3
speckyboy.com
Length 2
spectator.org
Length 8
speedcurve.com
Length 1
spendmatters.com
Length 2
spideroak.com
Length 1
spiffe.io
Length 1
spinditty.com
Length 5
splc.org
Length 1
splinternews.com
Length 1
spockframework.org
Length 1
spoonuniversity.com
Length 1
spravochnaya.gq
Length 1
sprudge.com
Length 1
sputnikn

zbw.eu
Length 9
zellwk.com
Length 1
zeltser.com
Length 1
zenhabits.net
Length 1
zenn.dev
Length 2
zerodha.com
Length 1
zone41.org
Length 2
zyro.com
Length 1


In [72]:
sim_dict["db-ip.com"]

url,http://db-ip.com/105.114.245.97,http://db-ip.com/139.181.9.0,http://db-ip.com/14.176.111.225,http://db-ip.com/148.221.0.0,http://db-ip.com/159.137.66.28,http://db-ip.com/159.142.246.54,http://db-ip.com/162.87.21.237,http://db-ip.com/171.10.130.148,http://db-ip.com/173.198.128.0,http://db-ip.com/177.124.72.0,...,http://db-ip.com/178.128.82.90,http://db-ip.com/181.150.110.101,http://db-ip.com/188.35.201.208,http://db-ip.com/188.40.236.91,http://db-ip.com/198.38.49.138,http://db-ip.com/2600:5c01:3050::,http://db-ip.com/68.37.93.27,http://db-ip.com/69.76.112.0,http://db-ip.com/7.170.206.174,http://db-ip.com/81.164.186.132
url,,,,,,,,,,,,,,,,,,,,,
http://db-ip.com/105.114.245.97,1.000000,0.991125,0.986560,0.995783,0.999997,0.995519,0.997055,0.991125,0.986351,0.990936,...,0.991125,0.990756,0.995648,0.982482,0.990756,0.990936,0.982482,0.990936,0.990756,0.986351
http://db-ip.com/139.181.9.0,0.991125,1.000000,0.999517,0.999136,0.991312,0.999253,0.998400,1.000000,0.999471,0.999998,...,1.000000,0.999991,0.999200,0.998507,0.999991,0.999998,0.998507,0.999998,0.999991,0.999471
http://db-ip.com/14.176.111.225,0.986560,0.999517,1.000000,0.997368,0.986798,0.997588,0.996182,0.999517,0.999998,0.999565,...,0.999517,0.999606,0.997484,0.999722,0.999606,0.999565,0.999722,0.999565,0.999606,0.999998
http://db-ip.com/148.221.0.0,0.995783,0.999136,0.997368,1.000000,0.995907,0.999990,0.999882,0.999136,0.997267,0.999072,...,0.999136,0.999007,0.999997,0.995388,0.999007,0.999072,0.995388,0.999072,0.999007,0.997267
http://db-ip.com/159.137.66.28,0.999997,0.991312,0.986798,0.995907,1.000000,0.995653,0.997163,0.991312,0.986594,0.991128,...,0.991312,0.990952,0.995777,0.982760,0.990952,0.991128,0.982760,0.991128,0.990952,0.986594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://db-ip.com/2600:5c01:3050::,0.990936,0.999998,0.999565,0.999072,0.991128,0.999198,0.998319,0.999998,0.999523,1.000000,...,0.999998,0.999998,0.999140,0.998596,0.999998,1.000000,0.998596,1.000000,0.999998,0.999523
http://db-ip.com/68.37.93.27,0.982482,0.998507,0.999722,0.995388,0.982760,0.995689,0.993867,0.998507,0.999755,0.998596,...,0.998507,0.998673,0.995547,1.000000,0.998673,0.998596,1.000000,0.998596,0.998673,0.999755
http://db-ip.com/69.76.112.0,0.990936,0.999998,0.999565,0.999072,0.991128,0.999198,0.998319,0.999998,0.999523,1.000000,...,0.999998,0.999998,0.999140,0.998596,0.999998,1.000000,0.998596,1.000000,0.999998,0.999523


In [24]:
cols = ['[document]',
 'a',
 'address',
 'ahref',
 'align',
 'app-root',
 'article',
 'aside',
 'av-box',
 'av-boxl',
 'avbody',
 'b',
 'b:if',
 'bdi',
 'bgcolor',
 'big',
 'blockquote',
 'body',
 'bodycopy',
 'bootstrap-card',
 'button',
 'c-wiz',
 'caption',
 'center',
 'channel',
 'cite',
 'clear',
 'code',
 'content',
 'dd',
 'ddp',
 'ddps',
 'del',
 'description',
 'details',
 'devsite-content',
 'dir',
 'div',
 'dl',
 'dt',
 'em',
 'fieldset',
 'figcaption',
 'figure',
 'font',
 'footer',
 'form',
 'h1',
 'h2',
 'h3',
 'h4',
 'h5',
 'h6',
 'head',
 'header',
 'html',
 'i',
 'index',
 'item',
 'label',
 'legend',
 'li',
 'main',
 'marquee',
 'nav',
 'noindex',
 'noscript',
 'novacuum',
 'o:p',
 'object',
 'ol',
 'p',
 'p:colorscheme',
 'p:onmouseclick',
 'phoenix-page',
 'pre',
 'projectcontent',
 'q',
 'rss',
 's',
 'section',
 'small',
 'span',
 'strong',
 'sub',
 'sup',
 'table',
 'tbody',
 'td',
 'template',
 'text',
 'textarea',
 'th',
 'thead',
 'tr',
 'tt',
 'u',
 'ul',
 'wbr',
 'whatuseek_winona_noindex']